# Лабораторная работа 5. Рекомендации по формализации и расчетам характеристик подсистем КИС в виде разомкнутых или замкнутых стохастических моделей

## Вариант 2

##### Выполнил: Борисочкин М. И. РТ5-81Б

In [ ]:
# Генерация начальных условий
Variant <- 2
set.seed(Variant)

m <- sample(c(4:18),1)
mu <- runif(1)
lambda <- runif(1)
if (lambda > mu)
{
  current <- lambda; 
   lambda <- mu; 
   mu <- current
}

gamma <- runif(1)
nu <- runif(1)
if (gamma < nu)
{
  current <- nu; 
  nu <- gamma; 
  gamma <- current
}

if (sample(c(0:1),1)) {k <- sample(c(4:7),1)} else {k <- "inf"}

t <- runif(1)
View(data.frame(lambda, mu, nu, gamma, k, m, t))

lambda,mu,nu,gamma,k,m,t
<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>
0.5733263,0.702374,0.1680519,0.9438393,4,8,0.8334488


### Теоритическая часть

Граф состояний:

![img](https://raw.githubusercontent.com/RDcool/BMSTU_AHP/main/Lab%204/State%20Graph.svg)

$S_0$ — Канал в рабочем состоянии, свободен;

$S_1$ — Канал в рабочем состоянии, обрабатывается заявка клиента, очереди нет;

$S_2$ — Канал в рабочем состоянии, обрабатывается заявка клиента, в очереди один человек;

$S_3$ — Канал в рабочем состоянии, обрабатывается заявка клиента, в очереди два человека;

$S_4$ — Канал в рабочем состоянии, обрабатывается заявка клиента, в очереди три человека;

$S_5$ — Канал вышел из строя, очереди нет;

$S_6$ — Канал вышел из строя, в очереди один человек;

$S_7$ — Канал вышел из строя, в очереди два человека;

$S_8$ — Канал вышел из строя, в очереди три человека;

$S_9$ — Канал вышел из строя, в очереди четыре человека.

Уравнения Колмогорова:

$
\begin{cases} 
\frac{dP_0(t)}{dt} = μP_1(t) - 4λP_0(t) + γP_5(t) - νP_0(t) \\
\frac{dP_1(t)}{dt} = 4λP_0(t) - μP_1(t) + μP_2(t) - 3λP_1(t) + γP_6(t) - νP_1(t) \\
\frac{dP_2(t)}{dt} = 3λP_1(t) - μP_2(t) + μP_3(t) - 2λP_2(t) + γP_7(t) - νP_2(t) \\
\frac{dP_3(t)}{dt} = 2λP_2(t) - μP_3(t) + μP_4(t) - λP_3(t) + γP_8(t) - νP_3(t) \\
\frac{dP_4(t)}{dt} = λP_3(t) - μP_4(t) + γP_9(t) - νP_4(t) \\
\frac{dP_5(t)}{dt} = νP_0(t) - γP_5(t) - 4λP_5(t)\\
\frac{dP_6(t)}{dt} = νP_1(t) - γP_6(t) + 4λP_5(t) - 3λP_6(t)\\
\frac{dP_7(t)}{dt} = νP_2(t) - γP_7(t) + 3λP_6(t) - 2λP_7(t)\\
\frac{dP_8(t)}{dt} = νP_3(t) - γP_8(t) + 2λP_7(t) - λP_8(t)\\
\frac{dP_9(t)}{dt} = νP_4(t) - γP_9(t) + λP_8(t)\\
\end{cases}
$

Заменим второе уравнение на условие нормировки:
$
\begin{cases} 
\frac{dP_0(t)}{dt} = μP_1(t) - 4λP_0(t) + γP_5(t) - νP_0(t) \\
1 = P_1(t) + P_2(t) + P_3(t) + P_4(t) + P_5(t) + P_6(t) + P_7(t) + P_8(t) + P_9(t)  \\
\frac{dP_2(t)}{dt} = 3λP_1(t) - μP_2(t) + μP_3(t) - 2λP_2(t) + γP_7(t) - νP_2(t) \\
\frac{dP_3(t)}{dt} = 2λP_2(t) - μP_3(t) + μP_4(t) - λP_3(t) + γP_8(t) - νP_3(t) \\
\frac{dP_4(t)}{dt} = λP_3(t) - μP_4(t) + γP_9(t) - νP_4(t) \\
\frac{dP_5(t)}{dt} = νP_0(t) - γP_5(t) - 4λP_5(t)\\
\frac{dP_6(t)}{dt} = νP_1(t) - γP_6(t) + 4λP_5(t) - 3λP_6(t)\\
\frac{dP_7(t)}{dt} = νP_2(t) - γP_7(t) + 3λP_6(t) - 2λP_7(t)\\
\frac{dP_8(t)}{dt} = νP_3(t) - γP_8(t) + 2λP_7(t) - λP_8(t)\\
\frac{dP_9(t)}{dt} = νP_4(t) - γP_9(t) + λP_8(t)\\
\end{cases}
$

Для решения данной системы уравнений (а если точнее нахождения вероятностей для заданного t) будем использовать библиотеку [deSolve](https://cran.r-project.org/web/packages/deSolve/deSolve.pdf)

In [ ]:
# Установка библиотеки
install.packages("deSolve")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
# Подключение библиотеки
library("deSolve")

In [ ]:
# Описание системы уравнений
Kolmogorovs <- function(t, state, parameters) {
  with(as.list(c(state, parameters)), {
    dP0 <- mu*P1 - 4*lambda*P0 + gamma*P5 - nu*P0
    dP1 <- 4*lambda*P0 - mu*P1 + mu*P2 - 3*lambda*P1 + gamma*P6 - nu*P1
    dP2 <- 3*lambda*P1 - mu*P2 + mu*P3 - 2*lambda*P2 + gamma*P7 - nu*P2
    dP3 <- 2*lambda*P2 - mu*P3 + mu*P4 - lambda*P3 + gamma*P8 - nu*P3
    dP4 <- lambda*P3 - mu*P4 + gamma*P9 - nu*P4
    dP5 <- nu*P0 - gamma*P5 - 4*lambda*P5
    dP6 <- nu*P1 - gamma*P6 + 4*lambda*P5 - 3*lambda*P6
    dP7 <- nu*P2 - gamma*P7 + 3*lambda*P6 - 2*lambda*P7
    dP8 <- nu*P3 - gamma*P8 + 2*lambda*P7 - lambda*P8
    dP9 <- nu*P4 - gamma*P9 + lambda*P8
    condNorm <- P0 + P1 + P2 + P3 + P4 + P5 + P6 + P7 + P8 + P9 

    list(c(dP0, dP1, dP2, dP3, dP4, dP5, dP6, dP7, dP8, dP9), condNorm = 1)
  })
}

In [ ]:
# Задание начального состояния и моментов времени (параметры определены ранее)
state <- c(P0 = 1, P1 = 0, P2 = 0, P3 = 0, P4 = 0, P5 = 0, P6 = 0, P7 = 0, P8 = 0, P9 = 0)
times <- seq(0, 1, by = 0.0000001)

In [ ]:
# Получение результатов
out <- ode(y = state, times = times, func = Kolmogorovs, parms = NULL)

In [ ]:
# Вывод вероятностей в момент времени t
outDF <- as.data.frame(out[, 1:11])
rownames(outDF) <- outDF$time

Ptau <- outDF[outDF$time == round(t, 7), 2:11]
Ptau

,P0,P1,P2,P3,P4,P5,P6,P7,P8,P9
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.8334488,0.2189755,0.3363448,0.2500563,0.09049219,0.01282055,0.01667962,0.03388733,0.02825842,0.01088262,0.001602681


In [ ]:
# Сумма вероятностей при заданном моменте t
sum(Ptau)

[1] 1

Для расчёта значений вероятностей при $t → ∞$  занулим призводные:

$
\begin{cases} 
μP_1(t) + γP_5(t) - (4λ + ν)P_0(t) = 0 \\
3λP_1(t) - (μ + 2λ + ν)P_2(t) + μP_3(t) + γP_7(t) = 0 \\
2λP_2(t) - (μ + λ + ν)P_3(t) + μP_4(t) + γP_8(t) = 0 \\
λP_3(t) - (μ + ν)P_4(t) + γP_9(t) = 0 \\
νP_0(t) - (γ + 4λ)P_5(t) = 0\\
νP_1(t) - (γ + 3λ)P_6(t) + 4λP_5(t) = 0\\
νP_2(t) - (γ + 2λ)P_7(t) + 3λP_6(t) = 0\\
νP_3(t) - (γ + λ)P_8(t) + 2λP_7(t) = 0\\
νP_4(t) - γP_9(t) + λP_8(t) = 0\\
P_1(t) + P_2(t) + P_3(t) + P_4(t) + P_5(t) + P_6(t) + P_7(t) + P_8(t) + P_9(t) = 1 \\
\end{cases}
$

Для решения данной системы уравнений воспользуемся функцией solve из R

In [ ]:
 # Матрица коэффициентов
A <- matrix(c( -(4*lambda + nu), mu, 0, 0, 0, gamma, 0, 0, 0, 0,
            0, 3*lambda, -(mu + 2*lambda + nu), mu, 0, 0, 0, gamma, 0, 0,
            0, 0, 2*lambda, -(mu + lambda + nu), mu, 0, 0, 0, gamma, 0,
            0, 0, 0, lambda, -(mu + nu), 0, 0, 0, 0, nu,
            nu, 0, 0, 0, 0, -(gamma +4*lambda),0, 0, 0, 0,
            0, nu, 0, 0, 0, 4*lambda, -(gamma + 3*lambda), 0, 0, 0,
            0, 0, nu, 0, 0, 0, 3*lambda, -(gamma + 2*lambda), 0, 0,
            0, 0, 0, nu, 0, 0, 0, 2*lambda, -(gamma + lambda), 0,
            0, 0, 0, 0, nu, 0, 0, 0, lambda, -gamma,
            1, 1, 1, 1, 1, 1, 1, 1, 1, 1), nrow = 10, byrow = TRUE)

# Вектор свободных членов
B <- c(rep(0, 9), 1)

# Решение уравнения
solInf <- solve(A, B)

# Формирование ответа 
solInfDF <- as.data.frame(solInf)
solInfDF <- t(solInfDF)
colnames(solInfDF) <- c("P0", "P1", "P2", "P3", "P4", "P5", "P6", "P7", "P8", "P9")
rownames(solInfDF) <- ""
solInfDF

,P0,P1,P2,P3,P4,P5,P6,P7,P8,P9
,0.0309372,0.1062563,0.2050391,0.297541,0.2090858,0.001606062,0.008086047,0.02313573,0.05044339,0.06786938


In [ ]:
# Сумма финальных вероятностей
sum(solInfDF)

[1] 1

Вероятноcть простоя — вероятность $P_0$

In [ ]:
# Вероятность простоя
solInfDF[1]

[1] 0.0309372

Вероятность образования очереди — вероятность обратная сумме вероятностей $P_0, P_1, P_5$

In [ ]:
# Вероятность образования очереди
1 - solInfDF[1] - solInfDF[2] - solInfDF[6]

[1] 0.8612005

Так как количесвто клиентов, которых обслуживает серевер, меньше значения максимальной длины очереди, то абсолютная пропуская спобосность системы будет равна интенсивности входного потока $λ$

In [ ]:
# Абсолютная пропускная способность
absLambda <- lambda
absLambda

[1] 0.5733263

In [ ]:
# Cреднее число заявок в системе
queueSize <- c(1:4)

Lsys <- sum(solInf[3:5]*queueSize[1:3]) + sum(solInf[7:10]*queueSize)
Lsys

[1] 1.904544

Для расчёта среднего время нахождения заявок в системе нам понадобиться вначале посчитать среднее число заявок в системе

In [ ]:
# Cреднее время нахождения заявок в системе
L <- (k - Lsys)*lambda

Tsys <- Lsys/L
Tsys

[1] 1.585296

In [ ]:
# Средняя длина очереди
Lq <- Lsys - L/mu
Lq

[1] 0.1940871

In [ ]:
# Cреднее время нахождения в очереди
Wq <- Lq/L
Wq

[1] 0.1615535

### Экспериментальная часть

In [ ]:
# Задание начального состояния и моментов времени (параметры определены ранее)
n <- 45 # Максимальное значение
step <- 0.002 # Шаг

expState <- c(P0 = 1, P1 = 0, P2 = 0, P3 = 0, P4 = 0, P5 = 0, P6 = 0, P7 = 0, P8 = 0, P9 = 0)
expTimes <- seq(0, n, by = step)

In [ ]:
# Получение вероятностей для эксперимента
outExp <- ode(y = expState, times = expTimes, func = Kolmogorovs, parms = NULL)

In [ ]:
# Функция выбора состояния
chooseState <- function(outExpRow)
{
  resState <- NaN

  p <- runif(1)

  if(p < outExpRow[1]) resState <- "S0"
  else if (p >= sum(outExpRow[1:10] & p <= 1)) resState <- "S9"
  else 
  {
    for(j in 1 : 9)
    if(p >= sum(outExpRow[1:j]) & p < sum(outExpRow[1:(j + 1)]))
    {
      resState <- paste("S", j, sep = "", collapse = NULL)
      break
    } 
  }

  return(resState)
}

In [ ]:
# Проведение эксперимента
resExpData <- data.frame()

for(i in 1 : as.integer(n/step))
{
  curState <- chooseState(outExp[i,2:11])
  resExpData[i, 1] <- outExp[i, 1]
  resExpData[i, 2] <- curState

  resExpData[i, 3] <- switch(curState, "S0" = 0, "S1" = 0, "S2" = 1, "S3" = 2, "S4" = 3, "S5" = 0, "S6" = 1, "S7" = 2, "S8" = 3, "S9" = 4)

  if(curState == "S0") resExpData[i, 4] <- "idle"
  else if(curState == "S1" | curState == "S2" | curState == "S3" | curState == "S4") resExpData[i, 4] <- "working"
  else resExpData[i, 4] <- "broken"
}

colnames(resExpData) <- c("time", "state", "queue size", "status")


In [ ]:
# Вероятность простоя
sum(resExpData$state == "S0")/length(resExpData$state)

[1] 0.03115556

In [ ]:
# Вероятность образования очереди
sum(resExpData$`queue size` > 0)/length(resExpData$state)

[1] 0.8902222

In [ ]:
# Абсолютная пропускная способность
sum(resExpData$status == "working")/length(resExpData$state)*lambda

[1] 0.4702805

In [ ]:
# Cреднее число заявок в системе
LsysExp <- sum(resExpData$`queue size`)/length(resExpData$state)
LsysExp

[1] 2.068222

In [ ]:
# Cреднее время нахождения заявок в системе
Lexp <- (k - LsysExp)*lambda

LsysExp/Lexp

[1] 1.867403

In [ ]:
# Средняя длина очереди
LqExp <- LsysExp - Lexp/mu
LqExp

[1] 0.4913714

In [ ]:
# Cреднее время нахождения в очереди
LqExp/Lexp

[1] 0.4436606